# Setup

This notebook shows a small example on what the model setup can do. We will demonstrate this on five metabolic models of Moraxella, H. Influenza, S. aureus, S. epidermidis and D. pigrum. 

In [ ]:
from ncmw.utils import get_models
from ncmw.setup_models import set_default_configs_and_snm3_medium, create_consistent_model, gapfill_medium, gapfill_model

In [2]:
models = get_models("models")

In [3]:
for model in models:
    print(f"Growth {model.id}: {model.slim_optimize()}")

Growth CarveMe_SNM_gapfilled_model: 0.7022727466098068
Growth himodel: 63.919604406865695
Growth Staphylococcus_epidermidis_ATCC_12228: 0.5746259500792624
Growth DP_83VPs_KB5: 0.282365392532248
Growth iYS854: 1.360224986987768


## Cleaning the model

There are typically several blocked reactions or mass charge violations within a metabolic model. Thus before continueing we remove these e.g. using the FastCC algorithm.

In [5]:
consistent_models =[]
reports = []
for model in models:
    consistent_model, report = create_consistent_model(model)
    consistent_models.append(consistent_model)
    reports.append(report)
    

In [5]:
reports[0]

,Blocked reactions,Metabolite formula problems,Mass charge balance violations
Original Model,246,0,122
Consistent model,0,0,117


In [6]:
for model in consistent_models:
    print(f"Growth {model.id}: {model.slim_optimize()}")

Growth CarveMe_SNM_gapfilled_model_consistent: 0.7022727466097843
Growth himodel_consistent: 63.919604406865524
Growth Staphylococcus_epidermidis_ATCC_12228_consistent: 0.574625950079267
Growth DP_83VPs_KB5_consistent: 0.2823653925322485
Growth iYS854_consistent: 1.3602249869877714


## Set default configurations

This package specially focus on work within the nasal mikrobiome represented through the **SNM3** medium. Unfortunallty most metabolic are created for the gut thus often fail to achive growth on SNM3 as we can see here 3 of the 5 models do not achive growth.

In [7]:
consistent_snm3_models =[]
for model in consistent_models:
    consistent_snm3_models.append(set_default_configs_and_snm3_medium(model))

In [7]:
for model in consistent_snm3_models:
    print(f"Growth {model.id}: {model.slim_optimize()}")

Growth CarveMe_SNM_gapfilled_model_consistent: -2.0804218484030174e-15
Growth himodel_consistent: -2.0391293932942178e-14
Growth Staphylococcus_epidermidis_ATCC_12228_consistent: 0.574625950079267
Growth DP_83VPs_KB5_consistent: 4.55767014689217e-16
Growth iYS854_consistent: 2.558694612613423


## Gapfilling

While not necessarily required for community modeling, it may be beneficial for analysis if the models can achive growth at their own. We implement two strategies:
* Gapfill medium: From the perspective of a community we can see the medium as less strict, because different metabolites could be provided by other members of the community that are currently not modeled. Thus we can extend the medium by the minimimum number of metabolites such that the organism can grow.
* Gapfill model: We can add reactions to the model such that it can grow on the given medium.

In [8]:
for model in consistent_snm3_models:
    m2, extension = gapfill_medium(model)
    print(f"We need to add following metabolites to the medium: {extension}")
    print(f"Growth {m2.id}: {m2.slim_optimize()}")

We need to add following metabolites to the medium: ['EX_fe3_e']
Growth CarveMe_SNM_gapfilled_model_consistent: 1.2060771083081372
We need to add following metabolites to the medium: ['EX_23cump_e', 'EX_2pglyc_e', 'EX_ade_e', 'EX_chtbs_e', 'EX_mso3_e', 'EX_skm_e']
Growth himodel_consistent: 2.0940011524149797
We need to add following metabolites to the medium: []
Growth Staphylococcus_epidermidis_ATCC_12228_consistent: 0.574625950079267
We need to add following metabolites to the medium: ['EX_ile__L_e', 'EX_met__L_e', 'EX_26dap__M_e']
Growth DP_83VPs_KB5_consistent: 0.2823653925322478
We need to add following metabolites to the medium: []
Growth iYS854_consistent: 2.558694612613423


In [9]:
for model in consistent_snm3_models:
    m2, extension = gapfill_model(model)
    print(f"We need to add following reactions to the model: {[e.id for e in extension]}")
    print(f"Growth {m2.id}: {m2.slim_optimize()}")

/mnt/c/Users/manug/OneDrive/Uni/HIWI/ncmw/ncmw/setup_models/setup.py:63: UserWarning: The model still has no growth... . We try an alternative
  warn("The model still has no growth... . We try an alternative")


We need to add following reactions to the model: ['FE2tex', 'EX_fe2_e', 'FEROpp', 'O2tpp']
Growth CarveMe_SNM_gapfilled_model_consistent: 1.2058013624004358
We need to add following reactions to the model: ['SHK3Dr', 'PPM2']
Growth himodel_consistent: 1.2794310343289401
We need to add following reactions to the model: []
Growth Staphylococcus_epidermidis_ATCC_12228_consistent: 0.5746259500762596
We need to add following reactions to the model: ['ACHBS', 'AHSERL2', 'ASADi', 'ASPK', 'DHAD2', 'HSERTA', 'KARA2', 'UAAGMDt', 'EX_uaagmda_e', 'HSDy']
Growth DP_83VPs_KB5_consistent: 0.30044661945216933
We need to add following reactions to the model: []
Growth iYS854_consistent: 2.5586946126133348
